In [1]:
import json
import pandas as pd
import numpy as np
from datasets import load_dataset
from FlagEmbedding import BGEM3FlagModel
from FlagEmbedding import AbsEmbedder

#### 1、模型加载及向量库生成

In [2]:
### 模型加载
path="./huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181"
### 替换为实际的BGE下载的地址
model = BGEM3FlagModel(path,pooling_method='cls',use_fp16=False,return_sparse = True,trust_remote_code=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [3]:
### 批量向量化并入库
import pandas as pd
data = pd.read_csv("./law_faq.csv")
data=data.dropna()

query_list=list(data["title"])
reply=list(data["reply"])

from faiss_index import Faissindex
#  采用暴力检索的方式，即计算查询向量与所有数据库向量之间的距离，然后返回相似度最高的前k个向量。
#  归一化vectors
knowledge_embedder=model.encode(reply)

vectors=knowledge_embedder['dense_vecs']

sl_fassi=Faissindex(1024)
sl_fassi.create(vectors)

pre tokenize: 100%|██████████| 72/72 [00:02<00:00, 26.50it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 72/72 [01:08<00:00,  1.06it/s]


In [4]:
### 检索的例子
### 对于i的结果进行测试：
i=0
print(query_list[i])
print(reply[i])

vectors_single_test=np.reshape(model.encode(query_list[i]).get('dense_vecs'), (1, -1))
scores_test,index_test=sl_fassi.search(vectors_single_test,5)

print(index_test)
topk_result = data["reply"].values[index_test[0].tolist()]
topk_result

在法律中定金与订金的区别订金和定金哪个受
“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担保方式，目的在于促使债务人履行债务，保障债权人的债权得以实现。签合同时，对定金必需以书面形式进行约定，同时还应约定定金的数额和交付期限。给付定金一方如果不履行债务，无权要求另一方返还定金；接受定金的一方如果不履行债务，需向另一方双倍返还债务。债务人履行债务后，依照约定，定金应抵作价款或者收回。而“订金”目前我国法律没有明确规定，它不具备定金所具有的担保性质，可视为“预付款”，当合同不能履行时，除不可抗力外，应根据双方当事人的过错承担违约责任。
[[ 5649  7522 10433 10574  9666]]


array(['我国法律认可”定金”，不认可”订金”。法律意义上:定金是指付款的担保，比如在买卖关系中，买*在签订合同后付给卖*部分款项，作为将来付款的担保，如果买*不履行合同，卖*可以不退还定金，如果卖*按照合同履行了供货义务，定金便折为货款，如果卖*没有按照合同履行供货义务，则应当承担双倍返还定金的责任。总而言之，定金是担保之意，也是法律中的用语。订金是一般被视为预付款，即使认定为一种履约保证，这种保证也是单*的，它只对给付*形成约束，即给付*对收受*的保证。若收受*违约，只能退回原订金，得不到双倍返还；若给付*违约，收受*会以种种理由把订金抵作赔偿金或违约金而不予退还。法律中，存在着定金罚则，而订金不适用该罚则。',
       '订金定金和订金虽然读音一样，但却有本质区别。“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担保方式，目的在于促使债务人履行债务，保障债权人的债权得以实现。签合同时，对定金必需以书面形式进行约定，同时还应约定定金的数额和交付期限。给付定金一方如果不履行债务，无权要求另一方返还定金；接受定金的一方如果不履行债务，需向另一方双倍返还债务。债务人履行债务后，依照约定，定金应抵作价款或者收回。而“订金”目前我国法律没有明确规定，它不具备定金所具有的担保性质，可视为“预付款”，当合同不能履行时，除不可抗力外，应根据双方当事人的过错承担违约责任。特别提示：并不是所有的“定金”都不能退还。我国《商品房销售管理办法》第22条规定：不符合商品房销售条件的，开发商不得销售商品房也不得收取任何预订款性质的费用。因此，如果商品房不符合销售条件，而购房者已经交纳了“定金”，那么无论双方是否约定“定金”退还事项，开发商都应无条件退还定金给购房者。此外，根据《最高人民法院关于审理商品房买卖合同纠纷案件适用法律若干问题的解释》第4条的规定，因不能归责于当事人双方的事由，导致商品房买卖合同未能订立的，出卖方也是应当返还定金的',
       '“定金”指为保证合同的履行，消费者预先向家具销售者（卖方）交纳一定数额的钱款。合同上是“定金”的，依据《合同法》相关规定，一方违约时，双方有约定的按照约定执行；如果无约定，家具销售者违约时，“定金”双倍返还；消费者违约时，“定金”不返还。“定金”的总额不得超过合同标的的20%。而对

#### 2、计算原模型的前30位的计算错误比例

In [5]:
### 计算结果：

from tqdm import tqdm
bench_select=[10,20,30]

for bench in  bench_select:
    wrong_index=[]
    
    for i, query_x in tqdm(enumerate(query_list), desc="Processing queries"):
        ## 进行检索：
        vectors_single=model.encode(query_x).get('dense_vecs')
        vectors_single=np.reshape(vectors_single, (1, -1))
        scores, indexes=sl_fassi.search(vectors_single,bench)
        is_corr=0
        for index in indexes[0]:
            if bool(index==i):
                is_corr=is_corr+1
        if is_corr==0:
            wrong_index.append(i)
            
    print(f"Bench为{bench}的情况下，错误比例为：{len(wrong_index)/len(query_list)*100}%")

Processing queries: 18208it [05:17, 57.32it/s]


Bench为10的情况下，错误比例为：55.89850615114236%


Processing queries: 18208it [05:08, 59.00it/s]


Bench为20的情况下，错误比例为：48.72583479789104%


Processing queries: 18208it [05:11, 58.51it/s]

Bench为30的情况下，错误比例为：44.172891036906854%


#### 3、困难负样本的构造

In [ ]:

#### 构造的思路为：对于30位排序以外的样本随机取20个作为困难负样本

from tqdm import tqdm
len_data=data.shape[0]

for i in tqdm(range(len_data), desc="Processing queries"):
    query=query_list[i]
    pos=[]
    pos.append(reply[i])

    vectorssingle=np.reshape(model.encode(query).get('dense_vecs'), (1, -1))
    scores_neg,index_neg=sl_fassi.search(vectorssingle,60)
    index_neg=list(index_neg[0])[30:]

    ## 预测正确的情况
    try:
        result_list=[int(x) for x in index_neg]
        result_list.remove(i)
        neg= list(data["reply"].values[result_list][0:20])
    except ValueError:
        result_list=[int(x) for x in index_neg]
        neg=list(data["reply"].values[result_list][0:20])
        
    with open("/root/autodl-tmp/2025-project/BGE/ana_data_2/all_samples.jsonl","a", encoding="utf-8") as f:
        record={"query":query,
                "pos":pos,
                "neg": neg}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")